## PRIMO 2022 ML Homework 4
Estimating the issue priority in the bug tracker

train.csv - обучающий набор
test.csv - тестовый набор
sample_submission.csv - образец итогового файла с предсказаниями. Обратите внимание, что вместо True и False здесь используются 1 и 0.

<u><b>summary</u></b> — краткое описание проблемы.
<u><b>description</u></b> — более подробное описание проблемы. Здесь могут быть использованы языки разметки Markdown и YouTrack Wiki.
<u><b>reporter</u></b> — создатель задачи. Обратите внимание, что поле содержит объект JSON, а не простое имя.
<u><b>created</u></b> — дата создания задачи в миллисекундах с 1 января 1970 г.
<u><b>customFields</u></b> — JSON-объект со вспомогательными данными о задаче: подсистема, тип, связанная версия продукта и т. д. Использование потенциально изменяемых полей при обучении модели может привести к "утечке" данных (data leakage). Использовать их не запрещено, но будьте осторожны и валидируйте свои модели.
<u><b>links</u></b> — JSON-объект со ссылками на другие задачи, если такие есть: какие задачи связаны с этой, какие задачи дублируются этой и т. д.
<u><b>is_high_priority</u></b> — целевая переменная: 1, если проблема имеет высокий приоритет, и 0 - в противном случае.
Блокнот с фрагментами кода, которые показывают, как начать работу с полями JSON в CSV можно найти на dl.

### Размышления
Summary и Description долго и сложно парсить. Трогать не буду
Reporter - может иметь значение, но вряд ли
Created - выглядит бесперспективно
CustomFields - выглядит очень перспективно
Links - выглядит странно, но распарсить можно также, как CustomFields - добавлю

Из моделей - быстрее и эффективней всего оказался Random Forest - в него всё загрузим и посмотрим - переступим порог или нет

In [1]:
import pandas as pd
import numpy as np
import json

np.set_printoptions(precision=3)

import warnings
warnings.simplefilter('ignore')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

In [2]:
train = pd.read_csv('train.csv')
train_df = train
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105365 entries, 0 to 105364
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                105365 non-null  object
 1   summary           105364 non-null  object
 2   description       103548 non-null  object
 3   reporter          105365 non-null  object
 4   created           105365 non-null  int64 
 5   customFields      105365 non-null  object
 6   links             105365 non-null  object
 7   is_high_priority  105365 non-null  bool  
dtypes: bool(1), int64(1), object(6)
memory usage: 5.7+ MB


## JSON

In [3]:
json_fields = pd.json_normalize(
    train_df.customFields.map(
        json.loads
    ).map(
        lambda x: {
            field['name']: field['value'] for field in x
        }
    )
)
json_fields.set_index(train_df.index, inplace=True)
dummy_type = json_fields['Type.name'].str.get_dummies()
dummy_subsystem = json_fields['State.name'].str.get_dummies()

In [4]:
json_links = pd.json_normalize(
    train_df.links.map(
        json.loads
    ).map(
        lambda x: {
            link['direction'] + link['linkType']['name']: len(link['issues']) for link in x
        }
    )
)
json_links.set_index(train_df.index, inplace=True)

## Random Forest

In [8]:
forest_X = pd.concat([json_links, dummy_type, dummy_subsystem], axis = 1)
if not ('Security (deprecated use Security Problem instead)' in forest_X.columns):
    forest_X['Security (deprecated use Security Problem instead)'] = 0
forest_Y = train_df['is_high_priority']

In [9]:
tree_threshold = .2

param_grid = dict(split_rand_state=range(5, 6), model_rand_state=range(5, 6), test_size=[0.2], n_estimators=[100], threshold=[tree_threshold])

forest_results = []

for params in tqdm(ParameterGrid(param_grid)):
    forest_X_train, forest_X_test, forest_Y_train, forest_Y_test = train_test_split(forest_X,
                                                                        forest_Y,
                                                                        stratify=forest_Y,
                                                                        random_state=params['split_rand_state'],
                                                                        test_size=params['test_size'])

    RF_model = RandomForestRegressor(random_state = params['model_rand_state'], n_estimators = params['n_estimators'])

    RF_model.fit(forest_X_train, forest_Y_train)

    forest_Y_train_predicted = RF_model.predict(forest_X_train)
    forest_Y_test_predicted = RF_model.predict(forest_X_test)

    forest_results.append(dict(
                        estimator=RF_model,
                        parameters=params,
                        train_f1 = f1_score(y_true=forest_Y_train, y_pred=forest_Y_train_predicted > params['threshold']),
                        test_f1 = f1_score(y_true=forest_Y_test, y_pred=forest_Y_test_predicted > params['threshold'])))

100%|██████████| 1/1 [00:26<00:00, 26.55s/it]


In [10]:
forest_result_df = pd.DataFrame(forest_results)
forest_result_df

,estimator,parameters,train_f1,test_f1
0,"(DecisionTreeRegressor(max_features='auto', ra...","{'model_rand_state': 5, 'n_estimators': 100, '...",0.327205,0.241642


## Test

In [11]:
test_df = pd.read_csv('test.csv')
test_df.summary.fillna('', inplace=True)
test_df.description.fillna('', inplace=True)

In [12]:
json_fields = pd.json_normalize(
    test_df.customFields.map(
        json.loads
    ).map(
        lambda x: {
            field['name']: field['value'] for field in x
        }
    )
)
json_fields.set_index(test_df.index, inplace=True)
dummy_type = json_fields['Type.name'].str.get_dummies()
dummy_subsystem = json_fields['State.name'].str.get_dummies()

json_links = pd.json_normalize(
    test_df.links.map(
        json.loads
    ).map(
        lambda x: {
            link['direction'] + link['linkType']['name']: len(link['issues']) for link in x
        }
    )
)
json_links.set_index(test_df.index, inplace=True)

forest_X = pd.concat([json_links, dummy_type, dummy_subsystem], axis = 1)

if not ('Security (deprecated use Security Problem instead)' in forest_X.columns):
    forest_X['Security (deprecated use Security Problem instead)'] = 0

In [13]:
forest_Y = RF_model.predict(forest_X)

In [14]:
probabilities = (forest_Y > tree_threshold).astype(int)

tree_result = pd.concat([test_df["id"], pd.DataFrame(probabilities, columns=['is_high_priority'])], axis=1)
tree_result.to_csv('PotatoTeam.csv', columns=['id', 'is_high_priority'], index=False)
tree_result

,id,is_high_priority
0,25-2512303,0
1,25-2513780,0
2,25-2515293,0
3,25-2514349,0
4,25-2514337,0
...,...,...
18524,25-3224524,0
18525,25-3224569,0
18526,25-3224564,0
18527,25-3224549,0
